# 🛠️ GitHubモデルを活用した高度なツール使用 (.NET)

## 📋 学習目標

このノートブックでは、Microsoft Agent Frameworkを使用してGitHubモデルと統合するエンタープライズ向けのツールパターンを紹介します。C#の強力な型付けと.NETのエンタープライズ機能を活用して、複数の専門ツールを持つ高度なエージェントを構築する方法を学びます。

**習得する高度なツール機能:**
- 🔧 **マルチツールアーキテクチャ**: 複数の専門的な機能を持つエージェントの構築
- 🎯 **型安全なツール実行**: C#のコンパイル時検証を活用
- 📊 **エンタープライズツールパターン**: 実運用向けのツール設計とエラー処理
- 🔗 **ツールの組み合わせ**: 複雑なビジネスワークフローのためのツール統合

## 🎯 .NETツールアーキテクチャの利点

### エンタープライズツールの特徴
- **コンパイル時検証**: 強力な型付けによりツールパラメータの正確性を保証
- **依存性注入**: IoCコンテナを使用したツール管理
- **非同期パターン**: 適切なリソース管理による非ブロッキングツール実行
- **構造化ログ**: ツール実行の監視のための組み込みログ機能

### 実運用向けパターン
- **例外処理**: 型付き例外による包括的なエラー管理
- **リソース管理**: 適切な破棄パターンとメモリ管理
- **パフォーマンス監視**: 組み込みのメトリクスとパフォーマンスカウンター
- **設定管理**: 型安全な設定とその検証

## 🔧 技術アーキテクチャ

### .NETツールの主要コンポーネント
- **Microsoft.Extensions.AI**: 統一されたツール抽象化レイヤー
- **Microsoft.Agents.AI**: エンタープライズ向けツールオーケストレーション
- **GitHubモデル統合**: 接続プールを備えた高性能APIクライアント

### ツール実行パイプライン
```csharp
User Request → Agent Analysis → Tool Selection → Type Validation
                 ↓               ↓              ↓
         Parameter Binding → Tool Execution → Result Processing → Response
```

## 🛠️ ツールカテゴリとパターン

### 1. **データ処理ツール**
- **入力検証**: データ注釈を使用した強力な型付け
- **変換操作**: 型安全なデータ変換とフォーマット
- **ビジネスロジック**: ドメイン固有の計算と分析ツール
- **出力フォーマット**: 構造化されたレスポンス生成

### 2. **統合ツール**
- **APIコネクタ**: HttpClientを使用したRESTfulサービス統合
- **データベースツール**: Entity Frameworkを使用したデータアクセス
- **ファイル操作**: 検証付きの安全なファイルシステム操作
- **外部サービス**: サードパーティサービス統合パターン

### 3. **ユーティリティツール**
- **テキスト処理**: 文字列操作とフォーマットユーティリティ
- **日付/時間操作**: 文化に配慮した日付/時間計算
- **数学ツール**: 精密な計算と統計操作
- **検証ツール**: ビジネスルールの検証とデータ確認

## ⚙️ 前提条件とセットアップ

**開発環境:**
- .NET 9.0 SDK以上
- Visual Studio 2022またはC#拡張機能付きVS Code
- GitHubモデルAPIアクセス

**必要なNuGetパッケージ:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**環境設定 (.envファイル):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

.NETで強力で型安全なツール機能を備えたエンタープライズ向けエージェントを構築する準備はできましたか？プロフェッショナルなソリューションを設計してみましょう！ 🏢⚡


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [ ]:
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [ ]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.2

In [4]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [5]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
using System;
using System.ComponentModel;
using System.ClientModel;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;

In [7]:
 using DotNetEnv;

In [8]:
Env.Load("../../../.env");

In [9]:
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan",
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
AgentThread thread = agent.GetNewThread();

In [15]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip",thread));

Your day trip will be in Paris, France! Here's a proposed itinerary for your day:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional French breakfast. Try a croissant and café au lait at a local café.
- **Visit the Eiffel Tower**: Arrive early to beat the crowds. Enjoy the stunning views of the city from the top.

### Late Morning
- **Stroll Along the Seine**: Take a leisurely walk along the Seine River and enjoy the picturesque views.
- **Visit Notre-Dame Cathedral**: Explore this iconic Gothic cathedral and its stunning architecture.

### Lunch
- **Lunch at a Bistro**: Treat yourself to a classic French lunch at a nearby bistro. Consider trying coq au vin or a delicious quiche.

### Afternoon
- **Explore the Louvre**: Spend your afternoon at the Louvre Museum. While you may not see everything, be sure to check out the Mona Lisa and other famous artworks.
- **Walk Through the Tuileries Garden**: After the museum, relax in the beautiful Tuileries Garden.

#

In [16]:
Console.WriteLine(await agent.RunAsync("I don't like that destination. Plan me another vacation.",thread));

Your new vacation destination is Rio de Janeiro, Brazil! Here's a proposed itinerary for your trip:

### Day 1: Arrival in Rio
- **Check-in**: Arrive and settle into your accommodation.
- **Evening at Copacabana Beach**: Stroll along the famous Copacabana Beach. Enjoy the vibrant atmosphere and watch the sunset.

### Day 2: Exploring the City
- **Visit Christ the Redeemer**: Start your day with a visit to the iconic Christ the Redeemer statue. Take the train up to the top for breathtaking views.
- **Explore Santa Teresa**: Wander through the charming streets of Santa Teresa, known for its colonial-style houses, art studios, and local shops.
- **Lunch at a Local Restaurant**: Enjoy traditional Brazilian cuisine, such as feijoada (black bean stew).
- **Afternoon at Sugarloaf Mountain**: Take a cable car ride to Sugarloaf Mountain for panoramic views of the city and coastline.

### Day 3: Cultural Experience
- **Visit the Selarón Steps**: Walk up the colorful Selarón Steps, an iconic mosa


---

**免責事項**:  
この文書は、AI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は一切の責任を負いません。
